In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
from Models.DeBruijn.Graph import Graph
from utils.converter import str2ncs, ncs2str

In [61]:
datas = ['GAGCGATGTCTC', 'GAGTGATGTTC', 'GAGTGTTGTCTC']
# datas = ['GAGTGTTGTCTC']
datas = ['TAATGCCATGGGATGTT']
datas = ['AAAGGCGTTGAGGTT']
datas = ['AACGGCGGCTGT', 'AACGGCGGCTGT', 'AACGGCGGCTGT', 'AACGGCGGCTGT']
# datas = ['AACGGCGGCTGT', 'AACGGCGACTGT', 'AACGGCGGCTGT', 'AACGGCGGCTGT']
# datas = ['AACGGCGGCTGT', 'AACGTGCGACTGT', 'AACGGCGCTGT', 'AACGGCGGCTGT']

graph = Graph(data = datas, data_length=12, kmer_size=4)
graph.build()
graph

Vertex AACG - 4.0(0, 4): ACGG 
Vertex ACGG - 4.0(4, 4): CGGC 
Vertex CGGC - 8.0(8, 8): GGCT -> GGCG 
Vertex GGCG - 4.0(4, 4): GCGG 
Vertex GCGG - 4.0(4, 4): CGGC 
Vertex GGCT - 4.0(4, 4): GCTG 
Vertex GCTG - 4.0(4, 4): CTGT 
Vertex CTGT - 4.0(4, 0): None 

In [65]:
graph.get_vertexes()

['AACG', 'ACGG', 'CGGC', 'GGCG', 'GCGG', 'GGCT', 'GCTG', 'CTGT']

In [74]:
ncs = str2ncs('Son')
print(ncs)
string = ncs2str('GAGTGATGTCTC')
print(string)

TGATGTCTC
PSON
